# Importing Libraries

In [4]:
import pandas as pd
import math
import numpy as np
from scipy.stats import norm
from scipy import stats

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
#from google.colab import drive
#drive.mount('/IMDB_data_NoDup.csv')

# Importing and reading csv movie dataset 
The size of the file is too big 350039x16. So will continue only with the 5000 data. 

In [6]:
IMDB_data_NoDup = "C:\\Users\\vishd\\Desktop\\reetu\\Springboard course\\Data wrangling IMDB dataset\\IMDB_data_NoDup.csv"
#IMDB_data_NoDup.head()
#IMDB_data_NoDup = "/content/IMDB_data_NoDup.csv"
#IMDB_data_NoDup = pd.read_csv(IMDB_data_NoDup, sep='\t')
#IMDB_data_NoDup.shape
IMDB_data_NoDup = pd.read_csv(IMDB_data_NoDup, sep='\t', nrows = 5000)#, delimiter=',', header=None, skiprows=1)#, names=['LABEL','F1','F2','F3','F4','F5','X','Y','Z','C1','C2'])                                                # loading first files
IMDB_data_NoDup.head()


,Index,titleId,title,region,titleType,startYear,genres,directors,writers,AveRating,Num_of_Votes,primaryName_Director,birthYear_Director,deathYear,primaryProfession_director,Dir_knownForTitles
0,0,tt0000019,The Clown Barber,GB,short,1898.0,"Comedy,Short",nm0932055,\N,5.3,19.0,James Williamson,1855.0,1933.0,"director,cinematographer,writer","tt0132534,tt0000355,tt0000273,tt0465763"
1,2,tt0271697,Our New Errand Boy,GB,short,1905.0,"Comedy,Short",nm0932055,\N,5.4,98.0,James Williamson,1855.0,1933.0,"director,cinematographer,writer","tt0132534,tt0000355,tt0000273,tt0465763"
2,5,tt0274177,Two Naughty Boys Teasing the Cobbler,GB,short,1898.0,"Comedy,Short",nm0932055,nm0932055,4.4,7.0,James Williamson,1855.0,1933.0,"director,cinematographer,writer","tt0132534,tt0000355,tt0000273,tt0465763"
3,14,tt0308848,Saved by a Dream,GB,short,1909.0,"Drama,Short",nm0932055,\N,6.3,6.0,James Williamson,1855.0,1933.0,"director,cinematographer,writer","tt0132534,tt0000355,tt0000273,tt0465763"
4,30,tt0454320,£100 Reward,GB,short,1908.0,"Drama,Short",nm0932055,\N,5.3,12.0,James Williamson,1855.0,1933.0,"director,cinematographer,writer","tt0132534,tt0000355,tt0000273,tt0465763"


# Droping one index col. 
Here we have two columns with index #. Need to drop index. 

In [7]:
IMDB_data_NoDup = IMDB_data_NoDup.drop(['Index'], axis = 1)                                             #dropping index column


# Basic Statistics 
Below is the basis statistics for our data set

In [8]:
IMDB_data_NoDup.describe()

,startYear,AveRating,Num_of_Votes
count,5000.000000,5000.000000,5000.000000
mean,1926.331600,5.806900,29.539600
std,18.794496,1.241449,45.563438
min,1894.000000,1.000000,5.000000
25%,1912.000000,5.000000,8.000000
50%,1926.000000,5.800000,13.000000
75%,1937.000000,6.600000,31.000000
max,2017.000000,10.000000,590.000000


# Converting years  to numeric columns

In [9]:
#converting year col ot numeric type

In [10]:
IMDB_data_NoDup['startYear'] = pd.to_numeric(IMDB_data_NoDup['startYear'], errors='coerce')
IMDB_data_NoDup['startYear'] = IMDB_data_NoDup["startYear"].convert_dtypes() 

IMDB_data_NoDup['startYear'].unique()

<IntegerArray>
[1898, 1905, 1909, 1908, 1906, 1900, 1901, 1897, 1895, 1896, 1894, 1903, 1899,
 1907, 1910, 1904, 1902, 1913, 1911, 1915, 1912, 1920, 1914, 1923, 1921, 1919,
 1922, 1936, 1916, 1917, 1926, 1932, 1933, 1928, 1927, 1929, 1931, 1935, 1930,
 1918, 1946, 1937, 1938, 1925, 1942, 1944, 1934, 1939, 1924, 2017, 1956, 1945,
 1950, 1940, 1943, 1953, 1954, 1957, 1959, 1960, 1968, 1962, 1964, 1965, 1967,
 1961, 1963, 1969, 1970, 1972, 1966, 1941, 1952, 1955, 1958, 1947, 1948, 1949,
 1951, 2011, 1974, 1973, 2004, 1999, 1971, 1975, 2014, 1977, 1978, 1981, 1982,
 1984, 2005]
Length: 93, dtype: Int64

In [11]:
#Converting death year col to numeric

In [12]:
IMDB_data_NoDup['deathYear'] = pd.to_numeric(IMDB_data_NoDup["deathYear"], errors='coerce')
IMDB_data_NoDup['deathYear'] = IMDB_data_NoDup["deathYear" ].convert_dtypes() 

In [13]:
# converting birthyear_Director col ot Numeric

In [14]:
IMDB_data_NoDup['birthYear_Director'] = pd.to_numeric(IMDB_data_NoDup["birthYear_Director"], errors='coerce')
IMDB_data_NoDup['birthYear_Director'] = IMDB_data_NoDup["birthYear_Director"].convert_dtypes() 


In [15]:
IMDB_data_NoDup.head()

,titleId,title,region,titleType,startYear,genres,directors,writers,AveRating,Num_of_Votes,primaryName_Director,birthYear_Director,deathYear,primaryProfession_director,Dir_knownForTitles
0,tt0000019,The Clown Barber,GB,short,1898,"Comedy,Short",nm0932055,\N,5.3,19.0,James Williamson,1855,1933,"director,cinematographer,writer","tt0132534,tt0000355,tt0000273,tt0465763"
1,tt0271697,Our New Errand Boy,GB,short,1905,"Comedy,Short",nm0932055,\N,5.4,98.0,James Williamson,1855,1933,"director,cinematographer,writer","tt0132534,tt0000355,tt0000273,tt0465763"
2,tt0274177,Two Naughty Boys Teasing the Cobbler,GB,short,1898,"Comedy,Short",nm0932055,nm0932055,4.4,7.0,James Williamson,1855,1933,"director,cinematographer,writer","tt0132534,tt0000355,tt0000273,tt0465763"
3,tt0308848,Saved by a Dream,GB,short,1909,"Drama,Short",nm0932055,\N,6.3,6.0,James Williamson,1855,1933,"director,cinematographer,writer","tt0132534,tt0000355,tt0000273,tt0465763"
4,tt0454320,£100 Reward,GB,short,1908,"Drama,Short",nm0932055,\N,5.3,12.0,James Williamson,1855,1933,"director,cinematographer,writer","tt0132534,tt0000355,tt0000273,tt0465763"


In [16]:
# We can see the type of startYear is int

In [17]:
IMDB_data_NoDup['startYear'].dtype

Int64Dtype()

In [18]:
# Calculating the age of the movie

In [19]:
Cuurent_year = 2021
IMDB_data_NoDup['Age_of_movie'] = Cuurent_year - IMDB_data_NoDup['startYear'] 

In [20]:
IMDB_data_NoDup.head()

,titleId,title,region,titleType,startYear,genres,directors,writers,AveRating,Num_of_Votes,primaryName_Director,birthYear_Director,deathYear,primaryProfession_director,Dir_knownForTitles,Age_of_movie
0,tt0000019,The Clown Barber,GB,short,1898,"Comedy,Short",nm0932055,\N,5.3,19.0,James Williamson,1855,1933,"director,cinematographer,writer","tt0132534,tt0000355,tt0000273,tt0465763",123
1,tt0271697,Our New Errand Boy,GB,short,1905,"Comedy,Short",nm0932055,\N,5.4,98.0,James Williamson,1855,1933,"director,cinematographer,writer","tt0132534,tt0000355,tt0000273,tt0465763",116
2,tt0274177,Two Naughty Boys Teasing the Cobbler,GB,short,1898,"Comedy,Short",nm0932055,nm0932055,4.4,7.0,James Williamson,1855,1933,"director,cinematographer,writer","tt0132534,tt0000355,tt0000273,tt0465763",123
3,tt0308848,Saved by a Dream,GB,short,1909,"Drama,Short",nm0932055,\N,6.3,6.0,James Williamson,1855,1933,"director,cinematographer,writer","tt0132534,tt0000355,tt0000273,tt0465763",112
4,tt0454320,£100 Reward,GB,short,1908,"Drama,Short",nm0932055,\N,5.3,12.0,James Williamson,1855,1933,"director,cinematographer,writer","tt0132534,tt0000355,tt0000273,tt0465763",113


In [21]:
IMDB_data_NoDup['Age_of_movie'].unique() 

<IntegerArray>
[123, 116, 112, 113, 115, 121, 120, 124, 126, 125, 127, 118, 122, 114, 111,
 117, 119, 108, 110, 106, 109, 101, 107,  98, 100, 102,  99,  85, 105, 104,
  95,  89,  88,  93,  94,  92,  90,  86,  91, 103,  75,  84,  83,  96,  79,
  77,  87,  82,  97,   4,  65,  76,  71,  81,  78,  68,  67,  64,  62,  61,
  53,  59,  57,  56,  54,  60,  58,  52,  51,  49,  55,  80,  69,  66,  63,
  74,  73,  72,  70,  10,  47,  48,  17,  22,  50,  46,   7,  44,  43,  40,
  39,  37,  16]
Length: 93, dtype: Int64

In [22]:
IMDB_data_NoDup['birthYear_Director'].unique()

<IntegerArray>
[1855, 1854, 1847, 1861, <NA>, 1860, 1873, 1872, 1869, 1874, 1875, 1870, 1864,
 1859, 1851, 1862, 1884, 1880, 1871, 1877, 1868, 1858, 1852, 1867, 1886, 1879,
 1876, 1863, 1885, 1881, 1853, 1878, 1883, 1865, 1882, 1888, 1887, 1857, 1891,
 1894, 1893, 1866, 1890, 1889, 1856, 1892, 1898, 1895, 1899, 1896, 1897, 1901,
 1902, 1900, 1903, 1904, 1908, 1906, 1910, 1905, 1907, 1909, 1911, 1922]
Length: 64, dtype: Int64

# Calculating the age of the directors

In [23]:
IMDB_data_NoDup['Director_age'] = IMDB_data_NoDup['deathYear']  - IMDB_data_NoDup['birthYear_Director'] 
IMDB_data_NoDup.head(2)

,titleId,title,region,titleType,startYear,genres,directors,writers,AveRating,Num_of_Votes,primaryName_Director,birthYear_Director,deathYear,primaryProfession_director,Dir_knownForTitles,Age_of_movie,Director_age
0,tt0000019,The Clown Barber,GB,short,1898,"Comedy,Short",nm0932055,\N,5.3,19.0,James Williamson,1855,1933,"director,cinematographer,writer","tt0132534,tt0000355,tt0000273,tt0465763",123,78
1,tt0271697,Our New Errand Boy,GB,short,1905,"Comedy,Short",nm0932055,\N,5.4,98.0,James Williamson,1855,1933,"director,cinematographer,writer","tt0132534,tt0000355,tt0000273,tt0465763",116,78


# Calculating std, min, max, mean of the rating columns 

In [24]:
std_Of_AvRating = np.std(IMDB_data_NoDup['AveRating'])
mean_mu = np.mean(IMDB_data_NoDup['AveRating'])
min_Rating = np.min(IMDB_data_NoDup['AveRating'])
max_Rating = np.max(IMDB_data_NoDup['AveRating'])
lowerbound, upperbound = stats.norm.interval(0.95, loc=mean_mu, scale= std_Of_AvRating)

print('std_Of_AvRating:',std_Of_AvRating, 'mean_mu of rating:', mean_mu)
print('min_Of_AvRating:',min_Rating, 'max_Of_AvRating:',min_Rating)
print('lowerbound:', lowerbound, 'upperbound', upperbound)

std_Of_AvRating: 1.2413244499324103 mean_mu of rating: 5.806899999999998
min_Of_AvRating: 1.0 max_Of_AvRating: 1.0
lowerbound: 3.3739487850034804 upperbound 8.239851214996516


In [25]:
#IMDB_data_NoDup['startYear'] = IMDB_data_NoDup['startYear'].astype(str).replace('\.0', '', regex=True)
#IMDB_data_NoDup['startYear'] = IMDB_data_NoDup['startYear'].astype(str).replace('\N', '', regex=True)

# Checking and filling NaN values 

In [26]:
#Checking NaN values
IMDB_data_NoDup['Age_of_movie'].isna().sum()


0

In [27]:
#Converting Age_of_movie to float type
IMDB_data_NoDup['Age_of_movie'] = IMDB_data_NoDup['Age_of_movie'].astype('float')

In [28]:
#Replacing all NaN value with mean of the Age_of_movie column 
IMDB_data_NoDup['Age_of_movie'].fillna((IMDB_data_NoDup['Age_of_movie'].mean()), inplace=True)

In [29]:
IMDB_data_NoDup['Age_of_movie'].value_counts()

109.0    170
124.0    166
91.0     153
108.0    136
123.0    135
        ... 
22.0       1
17.0       1
46.0       1
37.0       1
10.0       1
Name: Age_of_movie, Length: 93, dtype: int64

# dividing Age_of_movie col to bins(decade) of the movies

In [30]:

conditions = [
    # (IMDB_data_NoDup['Age_of_movie'].ndim == 0),
     (IMDB_data_NoDup['Age_of_movie'] <= 1),
     (IMDB_data_NoDup['Age_of_movie'] > 1) & (IMDB_data_NoDup['Age_of_movie'] <= 10), 
     (IMDB_data_NoDup['Age_of_movie'] > 10) & (IMDB_data_NoDup[ 'Age_of_movie'] <= 25),
     (IMDB_data_NoDup['Age_of_movie'] > 25) & (IMDB_data_NoDup['Age_of_movie'] <= 50),
     (IMDB_data_NoDup['Age_of_movie'] > 50) & (IMDB_data_NoDup['Age_of_movie'] <= 75),
     (IMDB_data_NoDup['Age_of_movie'] > 75) & (IMDB_data_NoDup['Age_of_movie'] <= 100),
     (IMDB_data_NoDup['Age_of_movie'] > 100) & (IMDB_data_NoDup['Age_of_movie'] <= 125),
     (IMDB_data_NoDup['Age_of_movie'] > 125) & (IMDB_data_NoDup['Age_of_movie']<= 150),
     (IMDB_data_NoDup['Age_of_movie'] > 150)]

# create a list of the values we want to assign for each condition
values = ['D0','D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8']

IMDB_data_NoDup['Decade']  = np.select(conditions, values) 

IMDB_data_NoDup.head()
    

,titleId,title,region,titleType,startYear,genres,directors,writers,AveRating,Num_of_Votes,primaryName_Director,birthYear_Director,deathYear,primaryProfession_director,Dir_knownForTitles,Age_of_movie,Director_age,Decade
0,tt0000019,The Clown Barber,GB,short,1898,"Comedy,Short",nm0932055,\N,5.3,19.0,James Williamson,1855,1933,"director,cinematographer,writer","tt0132534,tt0000355,tt0000273,tt0465763",123.0,78,D6
1,tt0271697,Our New Errand Boy,GB,short,1905,"Comedy,Short",nm0932055,\N,5.4,98.0,James Williamson,1855,1933,"director,cinematographer,writer","tt0132534,tt0000355,tt0000273,tt0465763",116.0,78,D6
2,tt0274177,Two Naughty Boys Teasing the Cobbler,GB,short,1898,"Comedy,Short",nm0932055,nm0932055,4.4,7.0,James Williamson,1855,1933,"director,cinematographer,writer","tt0132534,tt0000355,tt0000273,tt0465763",123.0,78,D6
3,tt0308848,Saved by a Dream,GB,short,1909,"Drama,Short",nm0932055,\N,6.3,6.0,James Williamson,1855,1933,"director,cinematographer,writer","tt0132534,tt0000355,tt0000273,tt0465763",112.0,78,D6
4,tt0454320,£100 Reward,GB,short,1908,"Drama,Short",nm0932055,\N,5.3,12.0,James Williamson,1855,1933,"director,cinematographer,writer","tt0132534,tt0000355,tt0000273,tt0465763",113.0,78,D6


In [31]:
IMDB_data_NoDup.shape

(5000, 18)

In [32]:
IMDB_data_NoDup['Decade'].value_counts()

D5    2200
D6    1962
D4     753
D7      55
D3      21
D1       6
D2       3
Name: Decade, dtype: int64

In [33]:
IMDB_data_NoDup[IMDB_data_NoDup['Decade'] == '0']

,titleId,title,region,titleType,startYear,genres,directors,writers,AveRating,Num_of_Votes,primaryName_Director,birthYear_Director,deathYear,primaryProfession_director,Dir_knownForTitles,Age_of_movie,Director_age,Decade


In [34]:
#IMDB_data_NoDup['Genre','TitleType2'] = IMDB_data_NoDup.genres.str.split(",", expand=True)
#df[['First','Last']] = df.Name.str.split("_",expand=True)
#IMDB_data_NoDup['First','Last'] = IMDB_data_NoDup.genres.apply(lambda x: pd.Series(str(x).split(",")))
#IMDB_data_NoDup.head()

# Checking the info of the dataframe


In [35]:
IMDB_data_NoDup.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   titleId                     5000 non-null   object 
 1   title                       5000 non-null   object 
 2   region                      5000 non-null   object 
 3   titleType                   5000 non-null   object 
 4   startYear                   5000 non-null   Int64  
 5   genres                      5000 non-null   object 
 6   directors                   5000 non-null   object 
 7   writers                     5000 non-null   object 
 8   AveRating                   5000 non-null   float64
 9   Num_of_Votes                5000 non-null   float64
 10  primaryName_Director        5000 non-null   object 
 11  birthYear_Director          4910 non-null   Int64  
 12  deathYear                   4901 non-null   Int64  
 13  primaryProfession_director  5000 

In [36]:
print(IMDB_data_NoDup.isnull().values.sum())

299


In [37]:
print(IMDB_data_NoDup.isnull().sum())

titleId                         0
title                           0
region                          0
titleType                       0
startYear                       0
genres                          0
directors                       0
writers                         0
AveRating                       0
Num_of_Votes                    0
primaryName_Director            0
birthYear_Director             90
deathYear                      99
primaryProfession_director      0
Dir_knownForTitles              0
Age_of_movie                    0
Director_age                  110
Decade                          0
dtype: int64


In [38]:
 #IMDB_data_NoDup.fillna(IMDB_data_NoDup['primaryProfession_director'].value_counts().index[0])

In [39]:
print(IMDB_data_NoDup.isnull().values.sum())

299


In [40]:
# selecting the values by removing the duplicate value and selecting unique values on the basisi of 
#title", "startYear", "primaryName_Director", "birthYear_Director"

In [41]:
DF_uniquevalue = IMDB_data_NoDup.value_counts(["title", "startYear", "primaryName_Director", "birthYear_Director" ]) 
DF_uniquevalue                                                        #selection of dropping column

title                         startYear  primaryName_Director  birthYear_Director
Metro Movietone Revue         1930       Nick Grinde           1893                  2
Old Glory and the Cuban Flag  1898       William Heise         1847                  2
American Flag                 1898       William Heise         1847                  2
Screen Snapshots              1932       Ralph Staub           1899                  2
Metro Movietone Revue         1929       Nick Grinde           1893                  2
                                                                                    ..
He and She                    1898       William K.L. Dickson  1860                  1
He Was Her Man                1937       Friz Freleng          1904                  1
He Forgot to Remember         1944       Hal Yates             1899                  1
Hazafelé                      1940       Arzén von Cserépy     1881                  1
Üzenet a Volgapartról         1942       Alfréd 

# Dropping the duplicate cols after selecting the uniques values from each cols

In [42]:
IMDB_data_NoDup = IMDB_data_NoDup.drop_duplicates(subset=["title", "startYear", "primaryName_Director", "birthYear_Director"])
IMDB_data_NoDup.head()

,titleId,title,region,titleType,startYear,genres,directors,writers,AveRating,Num_of_Votes,primaryName_Director,birthYear_Director,deathYear,primaryProfession_director,Dir_knownForTitles,Age_of_movie,Director_age,Decade
0,tt0000019,The Clown Barber,GB,short,1898,"Comedy,Short",nm0932055,\N,5.3,19.0,James Williamson,1855,1933,"director,cinematographer,writer","tt0132534,tt0000355,tt0000273,tt0465763",123.0,78,D6
1,tt0271697,Our New Errand Boy,GB,short,1905,"Comedy,Short",nm0932055,\N,5.4,98.0,James Williamson,1855,1933,"director,cinematographer,writer","tt0132534,tt0000355,tt0000273,tt0465763",116.0,78,D6
2,tt0274177,Two Naughty Boys Teasing the Cobbler,GB,short,1898,"Comedy,Short",nm0932055,nm0932055,4.4,7.0,James Williamson,1855,1933,"director,cinematographer,writer","tt0132534,tt0000355,tt0000273,tt0465763",123.0,78,D6
3,tt0308848,Saved by a Dream,GB,short,1909,"Drama,Short",nm0932055,\N,6.3,6.0,James Williamson,1855,1933,"director,cinematographer,writer","tt0132534,tt0000355,tt0000273,tt0465763",112.0,78,D6
4,tt0454320,£100 Reward,GB,short,1908,"Drama,Short",nm0932055,\N,5.3,12.0,James Williamson,1855,1933,"director,cinematographer,writer","tt0132534,tt0000355,tt0000273,tt0465763",113.0,78,D6


In [43]:
IMDB_data_NoDup.shape

(4995, 18)

In [44]:
# verification of the duplicationg of the cols.

Ver_dup = IMDB_data_NoDup[IMDB_data_NoDup['title'] == "Pilot"]
Ver_dup

,titleId,title,region,titleType,startYear,genres,directors,writers,AveRating,Num_of_Votes,primaryName_Director,birthYear_Director,deathYear,primaryProfession_director,Dir_knownForTitles,Age_of_movie,Director_age,Decade


In [45]:
IMDB_data_NoDup['title'].value_counts().count()

4934

In [46]:
IMDB_data_NoDup.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4995 entries, 0 to 4999
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   titleId                     4995 non-null   object 
 1   title                       4995 non-null   object 
 2   region                      4995 non-null   object 
 3   titleType                   4995 non-null   object 
 4   startYear                   4995 non-null   Int64  
 5   genres                      4995 non-null   object 
 6   directors                   4995 non-null   object 
 7   writers                     4995 non-null   object 
 8   AveRating                   4995 non-null   float64
 9   Num_of_Votes                4995 non-null   float64
 10  primaryName_Director        4995 non-null   object 
 11  birthYear_Director          4905 non-null   Int64  
 12  deathYear                   4896 non-null   Int64  
 13  primaryProfession_director  4995 

In [47]:
print(IMDB_data_NoDup.isnull().sum())

titleId                         0
title                           0
region                          0
titleType                       0
startYear                       0
genres                          0
directors                       0
writers                         0
AveRating                       0
Num_of_Votes                    0
primaryName_Director            0
birthYear_Director             90
deathYear                      99
primaryProfession_director      0
Dir_knownForTitles              0
Age_of_movie                    0
Director_age                  110
Decade                          0
dtype: int64


In [48]:
b = IMDB_data_NoDup[IMDB_data_NoDup['title'] == "Pilot"]                       # Checking pilot(duplicate)
b.shape

(0, 18)

In [49]:
IMDB_data_NoDup_copy = IMDB_data_NoDup.copy()

In [50]:
IMDB_data_NoDup_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4995 entries, 0 to 4999
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   titleId                     4995 non-null   object 
 1   title                       4995 non-null   object 
 2   region                      4995 non-null   object 
 3   titleType                   4995 non-null   object 
 4   startYear                   4995 non-null   Int64  
 5   genres                      4995 non-null   object 
 6   directors                   4995 non-null   object 
 7   writers                     4995 non-null   object 
 8   AveRating                   4995 non-null   float64
 9   Num_of_Votes                4995 non-null   float64
 10  primaryName_Director        4995 non-null   object 
 11  birthYear_Director          4905 non-null   Int64  
 12  deathYear                   4896 non-null   Int64  
 13  primaryProfession_director  4995 

In [51]:
# We can see the missing values in birthYear_Director, deathYear and Director_age. In next step we will 
#replace all missing values with mean values of the respective columns

In [52]:
# checking the value and percentage of missing values

In [53]:
#IMDB_data_NoDup_copy[IMDB_data_NoDup_copy['startYear'].isnull().sum()
IMDB_data_NoDup_copy.isnull().sum()

titleId                         0
title                           0
region                          0
titleType                       0
startYear                       0
genres                          0
directors                       0
writers                         0
AveRating                       0
Num_of_Votes                    0
primaryName_Director            0
birthYear_Director             90
deathYear                      99
primaryProfession_director      0
Dir_knownForTitles              0
Age_of_movie                    0
Director_age                  110
Decade                          0
dtype: int64

In [54]:
IMDB_data_NoDup_copy.isna().sum()/(len(IMDB_data_NoDup_copy))*100

titleId                       0.000000
title                         0.000000
region                        0.000000
titleType                     0.000000
startYear                     0.000000
genres                        0.000000
directors                     0.000000
writers                       0.000000
AveRating                     0.000000
Num_of_Votes                  0.000000
primaryName_Director          0.000000
birthYear_Director            1.801802
deathYear                     1.981982
primaryProfession_director    0.000000
Dir_knownForTitles            0.000000
Age_of_movie                  0.000000
Director_age                  2.202202
Decade                        0.000000
dtype: float64

In [55]:
IMDB_data_NoDup_copy[IMDB_data_NoDup_copy['startYear'].isnull()].head(2)        # priting Coluumn with null values

,titleId,title,region,titleType,startYear,genres,directors,writers,AveRating,Num_of_Votes,primaryName_Director,birthYear_Director,deathYear,primaryProfession_director,Dir_knownForTitles,Age_of_movie,Director_age,Decade


In [56]:
IMDB_data_NoDup_copy[IMDB_data_NoDup_copy.isnull().any(axis=1)].head(2)      #Checking Null values in df

,titleId,title,region,titleType,startYear,genres,directors,writers,AveRating,Num_of_Votes,primaryName_Director,birthYear_Director,deathYear,primaryProfession_director,Dir_knownForTitles,Age_of_movie,Director_age,Decade
130,tt0000053,A Chegada do Comboio Inaugural à Estação Centr...,PT,short,1897,"Documentary,Short",nm0684607,\N,2.8,8.0,Francisco Pinto Moreira,<NA>,<NA>,director,"tt0138450,tt0000053,tt0138358",124.0,<NA>,D6
131,tt0138358,A Chegada do Rebocador 'Liberal' ao Porto de L...,PT,short,1897,"Documentary,Short",nm0684607,\N,4.9,7.0,Francisco Pinto Moreira,<NA>,<NA>,director,"tt0138450,tt0000053,tt0138358",124.0,<NA>,D6


In [57]:
 # replacing startyear Null value with mean
IMDB_data_NoDup_copy['startYear'].fillna((IMDB_data_NoDup_copy['startYear'].mean()), inplace=True)

In [58]:
# verifying mean Replacement

In [59]:
IMDB_data_NoDup_copy[IMDB_data_NoDup_copy['title'] == "The Wonderful World of Wilbur Pope"]    
IMDB_data_NoDup_copy[IMDB_data_NoDup_copy['title'] == "Barnyard WAAC"]  

,titleId,title,region,titleType,startYear,genres,directors,writers,AveRating,Num_of_Votes,primaryName_Director,birthYear_Director,deathYear,primaryProfession_director,Dir_knownForTitles,Age_of_movie,Director_age,Decade


In [60]:
# Checkin Null values in Column birthYear_Director 

In [61]:
IMDB_data_NoDup_copy[IMDB_data_NoDup_copy['birthYear_Director'].isnull()].head(2)   

,titleId,title,region,titleType,startYear,genres,directors,writers,AveRating,Num_of_Votes,primaryName_Director,birthYear_Director,deathYear,primaryProfession_director,Dir_knownForTitles,Age_of_movie,Director_age,Decade
130,tt0000053,A Chegada do Comboio Inaugural à Estação Centr...,PT,short,1897,"Documentary,Short",nm0684607,\N,2.8,8.0,Francisco Pinto Moreira,<NA>,<NA>,director,"tt0138450,tt0000053,tt0138358",124.0,<NA>,D6
131,tt0138358,A Chegada do Rebocador 'Liberal' ao Porto de L...,PT,short,1897,"Documentary,Short",nm0684607,\N,4.9,7.0,Francisco Pinto Moreira,<NA>,<NA>,director,"tt0138450,tt0000053,tt0138358",124.0,<NA>,D6


In [62]:
#Replacing Null values with mean
#IMDB_data_NoDup['startYear'].fillna((IMDB_data_NoDup['startYear'].mean()), inplace=True)

IMDB_data_NoDup_copy['birthYear_Director'] = IMDB_data_NoDup_copy['birthYear_Director'].astype('float')
IMDB_data_NoDup_copy['deathYear'] = IMDB_data_NoDup_copy['deathYear'].astype('float')
IMDB_data_NoDup_copy['Director_age'] = IMDB_data_NoDup_copy['Director_age'].astype('float')



IMDB_data_NoDup_copy['birthYear_Director'].fillna((IMDB_data_NoDup_copy['birthYear_Director'].mean()), inplace = True)
IMDB_data_NoDup_copy['deathYear'].fillna((IMDB_data_NoDup_copy['deathYear'].mean()), inplace = True)
IMDB_data_NoDup_copy['Director_age'].fillna((IMDB_data_NoDup_copy['Director_age'].mean()), inplace = True) 
 

In [63]:
# checking columns after fillna 
IMDB_data_NoDup_copy[IMDB_data_NoDup_copy['primaryName_Director'] == "Francisco Pinto Moreira"] 

,titleId,title,region,titleType,startYear,genres,directors,writers,AveRating,Num_of_Votes,primaryName_Director,birthYear_Director,deathYear,primaryProfession_director,Dir_knownForTitles,Age_of_movie,Director_age,Decade
130,tt0000053,A Chegada do Comboio Inaugural à Estação Centr...,PT,short,1897,"Documentary,Short",nm0684607,\N,2.8,8.0,Francisco Pinto Moreira,1885.482161,1958.044322,director,"tt0138450,tt0000053,tt0138358",124.0,72.595701,D6
131,tt0138358,A Chegada do Rebocador 'Liberal' ao Porto de L...,PT,short,1897,"Documentary,Short",nm0684607,\N,4.9,7.0,Francisco Pinto Moreira,1885.482161,1958.044322,director,"tt0138450,tt0000053,tt0138358",124.0,72.595701,D6


In [64]:
# Checking NUll values after replacemnet
    #IMDB_data_NoDup_copy[IMDB_data_NoDup_copy['startYear'].isnull()]
IMDB_data_NoDup_copy[IMDB_data_NoDup_copy.isnull().any(axis=1)].head(2)          


,titleId,title,region,titleType,startYear,genres,directors,writers,AveRating,Num_of_Votes,primaryName_Director,birthYear_Director,deathYear,primaryProfession_director,Dir_knownForTitles,Age_of_movie,Director_age,Decade


In [65]:
# checking value after replacing all null values
IMDB_data_NoDup_copy.info()                        

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4995 entries, 0 to 4999
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   titleId                     4995 non-null   object 
 1   title                       4995 non-null   object 
 2   region                      4995 non-null   object 
 3   titleType                   4995 non-null   object 
 4   startYear                   4995 non-null   Int64  
 5   genres                      4995 non-null   object 
 6   directors                   4995 non-null   object 
 7   writers                     4995 non-null   object 
 8   AveRating                   4995 non-null   float64
 9   Num_of_Votes                4995 non-null   float64
 10  primaryName_Director        4995 non-null   object 
 11  birthYear_Director          4995 non-null   float64
 12  deathYear                   4995 non-null   float64
 13  primaryProfession_director  4995 

In [66]:
# value count for duplicates 
IMDB_data_NoDup_copy['title'].value_counts() 

The Poor Fish                      3
Faust                              3
The Reckoning                      3
One Touch of Nature                3
Hamlet                             3
                                  ..
500 Miles on a Gallon of Gas       1
The Swamp Fox: Redcoat Strategy    1
A Night in Town                    1
Gags and Gals                      1
The Pit and the Pendulum           1
Name: title, Length: 4934, dtype: int64

In [69]:
IMDB_data_NoDup_copy.to_csv('IMDB_data_NoDup_copy_for_prepossing.csv')